In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [6]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [7]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_dev shape:", X_dev.shape)
print("Y_dev shape:", Y_dev.shape)

X_train shape: (784, 41000)
Y_train shape: (41000,)
X_dev shape: (784, 1000)
Y_dev shape: (1000,)


In [39]:
def init_params(hidden_nodes=128):
    # Layer 1: 784 inputs -> 128 hidden nodes
    W1 = np.random.randn(hidden_nodes, 784) * 0.01 
    b1 = np.zeros((hidden_nodes, 1))
    
    # Layer 2: 128 hidden nodes -> 10 output nodes (for 10 digits)
    W2 = np.random.randn(10, hidden_nodes) * 0.01 
    b2 = np.zeros((10, 1))
    
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    # This is the stabilized version, which is correct.
    A = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return A / np.sum(A, axis=0, keepdims=True)

def deriv_ReLU(Z):
    return Z > 0

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size  
    one_hot_Y = one_hot(Y)
    
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [40]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # Removed the noisy print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

# --- IMPROVEMENT 3: MONITOR DEV SET ACCURACY ---
# We will pass the dev set into gradient descent to check
# our accuracy on "unseen" data. This is how we
# know if our model is truly learning or just memorizing.
def gradient_descent(X, Y, X_dev, Y_dev, iterations, alpha):
    # Use the new init_params with 128 hidden nodes
    W1, b1, W2, b2 = init_params(hidden_nodes=128)
    
    # Lists to store accuracy for plotting
    train_acc_history = []
    dev_acc_history = []
    iteration_steps = []
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        # We check accuracy every 50 iterations
        # (and on the last iteration)
        if i % 50 == 0 or i == iterations - 1:
            # Get training accuracy
            train_predictions = get_predictions(A2)
            train_acc = get_accuracy(train_predictions, Y)
            train_acc_history.append(train_acc)
            
            # Get development (validation) accuracy
            # We only need a forward pass, no backprop
            _, _, _, A2_dev = forward_prop(W1, b1, W2, b2, X_dev)
            dev_predictions = get_predictions(A2_dev)
            dev_acc = get_accuracy(dev_predictions, Y_dev)
            dev_acc_history.append(dev_acc)
            
            iteration_steps.append(i)
            
            print(f"Iteration: {i}")
            print(f"Train Accuracy: {train_acc:.4f}")
            print(f"Dev Accuracy:   {dev_acc:.4f}")
            print("-" * 20)
            
    # We add iteration_steps to the return for plotting
    return W1, b1, W2, b2, train_acc_history, dev_acc_history, iteration_steps

In [41]:

iterations = 1000
alpha = 0.1

W1, b1, W2, b2, train_acc, dev_acc, steps = gradient_descent(
    X_train, Y_train, 
    X_dev, Y_dev, 
    iterations, alpha
)

Iteration: 0
Train Accuracy: 0.1306
Dev Accuracy:   0.1480
--------------------
Iteration: 50
Train Accuracy: 0.6164
Dev Accuracy:   0.5960
--------------------
Iteration: 100
Train Accuracy: 0.7890
Dev Accuracy:   0.7680
--------------------
Iteration: 150
Train Accuracy: 0.8458
Dev Accuracy:   0.8360
--------------------
Iteration: 200
Train Accuracy: 0.8700
Dev Accuracy:   0.8640
--------------------
Iteration: 250
Train Accuracy: 0.8836
Dev Accuracy:   0.8800
--------------------
Iteration: 300
Train Accuracy: 0.8910
Dev Accuracy:   0.8890
--------------------
Iteration: 350
Train Accuracy: 0.8967
Dev Accuracy:   0.8950
--------------------
Iteration: 400
Train Accuracy: 0.9008
Dev Accuracy:   0.8990
--------------------
Iteration: 450
Train Accuracy: 0.9043
Dev Accuracy:   0.9010
--------------------
Iteration: 500
Train Accuracy: 0.9070
Dev Accuracy:   0.9070
--------------------
Iteration: 550
Train Accuracy: 0.9100
Dev Accuracy:   0.9090
--------------------
Iteration: 600
Trai

In [42]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_data = np.array(test_data)

# Transpose the data
# The shape will be (784, 28000)
test_data = test_data.T

# --- VERY IMPORTANT: Normalize the test data ---
# We must use the same preprocessing as our training data
X_test = test_data / 255.

print("Test data shape:", X_test.shape)

Test data shape: (784, 28000)


In [43]:
# Run a forward pass on the entire test set using our trained parameters
_, _, _, A2_test = forward_prop(W1, b1, W2, b2, X_test)

# Get the final predictions (the index of the max probability)
test_predictions = get_predictions(A2_test)

print("Predictions array:", test_predictions)
print("Number of predictions:", test_predictions.size)

Predictions array: [2 0 9 ... 3 9 2]
Number of predictions: 28000


In [45]:
# Create the ImageId column
# np.arange(1, 28001) creates an array [1, 2, 3, ..., 28000]
image_ids = np.arange(1, test_predictions.size + 1)

# Create a pandas DataFrame with the required columns
submission = pd.DataFrame({
    "ImageId": image_ids,
    "Label": test_predictions
})

# Save the DataFrame to a CSV file.
# 'index=False' is crucial, or Kaggle will reject the file.
submission.to_csv("/kaggle/working/submission.csv", index=False)

print("Submission file created successfully!")
submission.head()

Submission file created successfully!


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
